In [1]:
import os
import logging
import time
import threading
import json
from datetime import datetime
from typing import List, Dict, Any
import httpx
from anthropic import Anthropic, RateLimitError
from openai import OpenAI
import google.generativeai as gemini
from dotenv import load_dotenv
from tenacity import retry, wait_exponential, stop_after_attempt, retry_if_exception_type

# Basic setup
logging.basicConfig(level=logging.INFO)
load_dotenv()


/Users/ceadams/Documents/onclaive/onclaive/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:

# Constants
CERT_PATH = '/opt/homebrew/etc/openssl@3/cert.pem'
OUTPUT_DIR = 'processed_outputs'

QUESTIONS = [
    "What is the purpose of the FHIR DaVinci PlanNet Implementation Guide?",
    "Who are the intended users and actors of the FHIR DaVinci PlanNet Implementation Guide?",
    "Are there one or more workflows defined in the FHIR DaVinci PDex Plan Net Implementation Guide? Please use all the information you know.",
    "What data is being exchanged in the FHIR DaVinci PDex Plan Net Implementation Guide and why?",
    "How is that data represented by the resources and profiles in the FHIR DaVinci PDex Plan Net Implementation Guide?",
    "What actions (REST/CRUD) or operations can be used in the FHIR DaVinci PDex Plan Net Implementation Guide?",
    "What are all the mandatory requirements and rules from the DaVinci PDex Plan Net Implementation Guide for compliant implementations?",
    "What are all the optional requirements and rules from the DaVinci PDex Plan Net Implementation Guide for compliant implementations?",
    "How would you create a test plan for the FHIR DaVinci PDex Plan Net Implementation Guide?"
]

# Configuration for different LLMs
CLAUDE_CONFIG = {
    "model_name": "claude-3-5-sonnet-20240620",
    "max_tokens": 8192,
    "temperature": 0.7,
    "system_prompt": """You are a seasoned Healthcare Integration Test Engineer analyzing a FHIR Implementation Guide.
                    Provide detailed, technically precise answers focused on implementation details."""
}

GPT_CONFIG = {
    "model_name": "gpt-4",
    "max_tokens": 2048,
    "temperature": 0.7,
    "system_prompt": """You are a seasoned Healthcare Integration Test Engineer analyzing a FHIR Implementation Guide.
                    Provide detailed, technically precise answers focused on implementation details."""
}

GEMINI_CONFIG = {
    "model_name": "models/gemini-1.5-pro-001",
    "max_tokens": 8192,
    "temperature": 0.7
}

# Rate limiting configurations
RATE_LIMITS = {
    "claude": {
        "requests_per_minute": 25,
        "max_requests_per_day": 5000,
        "delay_between_requests": 2
    },
    "gpt": {
        "requests_per_minute": 200,
        "max_requests_per_day": 10000,
        "delay_between_requests": 0.5
    },
    "gemini": {
        "requests_per_minute": 60,
        "max_requests_per_day": 60000,
        "delay_between_requests": 1
    }
}

def create_rate_limiter():
    """Create a rate limiter state dictionary for all APIs"""
    return {
        api: {
            'requests': [],
            'daily_requests': 0,
            'last_reset': time.time()
        }
        for api in RATE_LIMITS.keys()
    }

def check_rate_limits(rate_limiter: dict, api: str):
    """Check and wait if rate limits would be exceeded"""
    if api not in rate_limiter:
        raise ValueError(f"Unknown API: {api}")
        
    now = time.time()
    state = rate_limiter[api]
    limits = RATE_LIMITS[api]
    
    # Reset daily counts if needed
    day_seconds = 24 * 60 * 60
    if now - state['last_reset'] >= day_seconds:
        state['daily_requests'] = 0
        state['last_reset'] = now
    
    # Check daily limit
    if state['daily_requests'] >= limits['max_requests_per_day']:
        raise Exception(f"{api} daily request limit exceeded")
    
    # Remove old requests outside the current minute
    state['requests'] = [
        req_time for req_time in state['requests']
        if now - req_time < 60
    ]
    
    # Wait if at rate limit
    if len(state['requests']) >= limits['requests_per_minute']:
        sleep_time = 60 - (now - state['requests'][0])
        if sleep_time > 0:
            time.sleep(sleep_time)
        state['requests'] = state['requests'][1:]
    
    # Add minimum delay between requests
    if state['requests'] and now - state['requests'][-1] < limits['delay_between_requests']:
        time.sleep(limits['delay_between_requests'])
    
    # Record this request
    state['requests'].append(now)
    state['daily_requests'] += 1

def heartbeat(stop_event, start_time):
    """Print elapsed time periodically until stopped"""
    while not stop_event.is_set():
        elapsed = time.time() - start_time
        print(f"... still processing ({elapsed:.1f}s elapsed)")
        time.sleep(5)


In [3]:

def setup_clients():
    """Initialize clients for each LLM service"""
    try:
        # Claude setup
        verify_path = CERT_PATH if os.path.exists(CERT_PATH) else True
        http_client = httpx.Client(verify=verify_path, timeout=60.0)
        claude_client = Anthropic(
            api_key=os.getenv('ANTHROPIC_API_KEY'),
            http_client=http_client
        )
        
        # Gemini setup
        gemini.configure(api_key=os.getenv('GEMINI_API_KEY'))
        gemini_client = gemini.GenerativeModel(
            model_name=GEMINI_CONFIG["model_name"],
            generation_config={
                "max_output_tokens": GEMINI_CONFIG["max_tokens"],
                "temperature": GEMINI_CONFIG["temperature"]
            }
        )
        
        # OpenAI setup
        openai_client = OpenAI(
            api_key=os.getenv('OPENAI_API_KEY'),
            timeout=60.0
        )
        
        return {
            "claude": claude_client,
            "gemini": gemini_client,
            "gpt": openai_client
        }
        
    except Exception as e:
        logging.error(f"Error setting up clients: {str(e)}")
        raise


In [ ]:

@retry(
    wait=wait_exponential(multiplier=1, min=4, max=60),
    stop=stop_after_attempt(5),
    retry=retry_if_exception_type((RateLimitError, TimeoutError))
)
def ask_llm(client: Any, question: str, llm_type: str, rate_limiter: dict) -> str:
    """Ask question to specific LLM with rate limiting and retries"""
    stop_event = threading.Event()
    start_time = time.time()
    
    heartbeat_thread = threading.Thread(
        target=heartbeat,
        args=(stop_event, start_time)
    )
    heartbeat_thread.start()
    
    try:
        check_rate_limits(rate_limiter, llm_type)
        
        if llm_type == "claude":
            response = client.messages.create(
                model=CLAUDE_CONFIG["model_name"],
                messages=[{"role": "user", "content": question}],
                system=CLAUDE_CONFIG["system_prompt"],
                max_tokens=CLAUDE_CONFIG["max_tokens"]
            )
            return response.content[0].text
            
        elif llm_type == "gemini":
            response = client.generate_content(question)
            return response.text
            
        elif llm_type == "gpt":
            response = client.chat.completions.create(
                model=GPT_CONFIG["model_name"],
                messages=[
                    {"role": "system", "content": GPT_CONFIG["system_prompt"]},
                    {"role": "user", "content": question}
                ],
                max_tokens=GPT_CONFIG["max_tokens"]
            )
            return response.choices[0].message.content
            
    except Exception as e:
        logging.error(f"Error in {llm_type} request: {str(e)}")
        raise
    
    finally:
        stop_event.set()
        heartbeat_thread.join()

def save_results(results: Dict[str, Dict[str, str]], llm_type: str):
    """Save analysis results with timestamp"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_path = os.path.join(OUTPUT_DIR, f"{llm_type}_plannet_analysis_{timestamp}.json")
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    with open(output_path, 'w') as f:
        json.dump({
            "metadata": {
                "timestamp": timestamp,
                "llm_type": llm_type,
                "generation_date": datetime.now().isoformat()
            },
            "results": results
        }, f, indent=2)

def analyze_with_llm(llm_type: str, client: Any, rate_limiter: dict) -> Dict[str, str]:
    """Run analysis for all questions using specified LLM"""
    results = {}
    
    for question in QUESTIONS:
        print(f"\nAnalyzing question with {llm_type}: {question}")
        response = ask_llm(client, question, llm_type, rate_limiter)
        results[question] = response
        time.sleep(2)  # Small delay between questions
        
    save_results(results, llm_type)
    return results

def run_analysis():
    """Run analysis using all LLMs"""
    try:
        # Setup
        os.makedirs(OUTPUT_DIR, exist_ok=True)
        clients = setup_clients()
        rate_limiter = create_rate_limiter()
        
        # Process with each LLM
        results = {}
        for llm_type, client in clients.items():
            print(f"\nProcessing with {llm_type}...")
            results[llm_type] = analyze_with_llm(llm_type, client, rate_limiter)
            
        return results
        
    except Exception as e:
        logging.error(f"Error in analysis: {str(e)}")
        raise

In [ ]:
results = run_analysis()
print(f"\nAnalysis complete! Results saved in: {OUTPUT_DIR}")


Processing with claude...

Analyzing question with claude: What is the purpose of the FHIR DaVinci PlanNet Implementation Guide?
... still processing (0.0s elapsed)
... still processing (5.0s elapsed)


INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"



Analyzing question with claude: Who are the intended users and actors of the FHIR DaVinci PlanNet Implementation Guide?
... still processing (0.0s elapsed)
